In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [290]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import datetime as dt
import random as ran
from sklearn.metrics.pairwise import euclidean_distances as euc

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

from src.dbc import TimeDivisionKMeans
from src.dbc.utils import data_preprocessing, dimension_reduction

In [454]:
# 1. load_excel
data_path = "data/apt_1.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

# 2. data preprocessing
df = data_preprocessing(xlsx)

# 3. dimension_reduction
df = dimension_reduction(df)

# 4. day clustering
df = df[:24]
tdKMeans = TimeDivisionKMeans(df)

df.head()

,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,...,아파트1-102-901,아파트1-103-1905,아파트1-103-503,아파트1-103-1504,아파트1-103-606,아파트1-103-903,아파트1-103-1106,아파트1-103-705,아파트1-103-1505,아파트1-103-406
2018-05-01 00:00:00,0.341,0.121,0.245,0.009,0.085,0.154,0.161,0.311,0.358,0.177,...,0.372,0.181,0.322,0.374,0.571,0.453,0.161,0.127,0.259,0.364
2018-05-01 01:00:00,0.337,0.186,0.285,0.010,0.071,0.127,0.179,0.225,0.206,0.178,...,0.245,0.238,0.292,0.382,0.327,0.352,0.172,0.133,0.234,0.286
2018-05-01 02:00:00,0.324,0.214,0.487,0.009,0.061,0.139,0.182,0.221,0.150,0.152,...,0.245,0.250,0.287,0.320,0.359,0.366,0.242,0.219,0.197,0.368
2018-05-01 03:00:00,0.319,0.250,0.300,0.009,0.067,0.132,0.172,0.274,0.142,0.205,...,0.177,0.223,0.329,0.170,0.322,0.267,0.269,0.164,0.229,0.414
2018-05-01 04:00:00,0.235,0.199,0.107,0.009,0.047,0.131,0.153,0.255,0.139,0.169,...,0.129,0.171,0.301,0.168,0.351,0.255,0.170,0.132,0.198,0.318


In [455]:
tdKMeans.init_setting()

init_setting func start
init_setting func end


In [642]:
households_cluster = pd.DataFrame(columns=df.columns)
cluster_info = list()
mem_now_df = list()

# self data
K = 7
size = 3

# init_setting 에 들어가면 좋을 거 같다.
# run 에서 다시 건들일 없을거 같은 변수들
households_size = len(df.columns)
total_size = len(df)
division_size = round(total_size / size)
division_df = [df[_:_ + size] for _ in range(0,total_size, size)]

# run 함수에 들어갈 친구들
for division_round in range(0,division_size):
    _round = 0
    _early_stop_cnt = 0
    prev_clusters = None
    clusters = np.zeros(households_size)
    K_pattern = np.array([])
    
    while True:
        # 초기 K 선정
        if _round == 0:
            init_K = np.array([])
            K_pattern = np.array([])

            while len(init_K) < K:
                _K = ran.randint(0, households_size - 1)

                if _K not in init_K:
                    init_K = np.append(init_K, _K)

            now_df = division_df[division_round].copy().T
            mem_now_df.append(now_df)
            idxes = now_df.index

            for _K in init_K:
                idx = idxes[int(_K)]
                pattern = now_df.loc[idx].values
                K_pattern = np.append(
                    K_pattern,
                    pattern
                )
                
            K_pattern = K_pattern.reshape(-1,size)
        else:
            next_round_K_pattern = np.array([])

            for idx in range(0, K):
                next_round_K_pattern = np.append(
                    next_round_K_pattern,
                    (np.round(now_df[clusters == idx].mean() * 1000) / 1000)
                )
            next_round_K_pattern = next_round_K_pattern.reshape(-1, size)    
            K_pattern = next_round_K_pattern
        
        
        clusters = np.array([])

        for idx in now_df.index:
            
            test = now_df.loc[idx].values
            test = np.expand_dims(test, axis=0)
            cluster = euc(test, K_pattern).argmin()
            
            if idx == "아파트1-104-1208":
                print(test)
                print(K_pattern)
                print(euc(test, K_pattern))
                print(cluster)
                print("-----------")
            clusters = np.append(clusters, cluster)

        if (clusters == prev_clusters).all():
            _early_stop_cnt += 1
        else:
            _early_stop_cnt = 0
            
        if _early_stop_cnt >= 3:
            # 시간대 기여도 산출
            total_kwh = K_pattern.sum()
            K_kwh = K_pattern.sum(axis=1)

            contributions = np.round(K_kwh / total_kwh * 100)
            print("기여도: {}".format(contributions))
            
            households_cluster.loc[division_round] = clusters
            cluster_info.append([K_pattern, contributions])
            
            break
            
        prev_clusters = clusters
        _round += 1
        
households_cluster

[[0.009 0.01  0.009]]
[[0.335 0.19  0.143]
 [0.152 0.336 0.493]
 [0.615 0.243 0.217]
 [0.192 0.192 0.19 ]
 [0.297 0.299 0.204]
 [0.161 0.16  0.205]
 [0.452 0.286 0.283]]
[[0.39576761 0.60081694 0.68175435 0.31523642 0.45220571 0.28986204
  0.58949215]]
5
-----------
[[0.009 0.01  0.009]]
[[0.375 0.191 0.166]
 [0.334 0.403 0.598]
 [0.72  0.451 0.368]
 [0.207 0.202 0.195]
 [0.296 0.294 0.249]
 [0.126 0.119 0.133]
 [0.473 0.396 0.362]]
[[0.437454   0.77909884 0.91043012 0.33266199 0.46970736 0.20235118
  0.69921456]]
5
-----------
[[0.009 0.01  0.009]]
[[0.474 0.217 0.197]
 [0.334 0.403 0.598]
 [0.706 0.587 0.448]
 [0.199 0.2   0.203]
 [0.32  0.298 0.274]
 [0.119 0.103 0.107]
 [0.48  0.373 0.355]]
[[0.54260299 0.77909884 1.00571318 0.33141515 0.49988999 0.17422112
  0.68798692]]
5
-----------
[[0.009 0.01  0.009]]
[[0.505 0.249 0.224]
 [0.198 0.31  0.49 ]
 [0.716 0.612 0.504]
 [0.2   0.196 0.2  ]
 [0.327 0.304 0.278]
 [0.115 0.095 0.093]
 [0.511 0.413 0.39 ]]
[[0.59106852 0.59756339 1.052

[[0.009 0.009 0.01 ]]
[[0.283 0.326 0.322]
 [0.314 0.426 0.542]
 [0.128 0.109 0.115]
 [0.507 0.436 0.369]
 [0.494 0.755 0.501]
 [0.287 0.219 0.21 ]
 [0.935 0.466 0.41 ]]
[[0.52240693 0.74157805 0.18757932 0.74780612 1.01627851 0.40172627
  1.10739559]]
2
-----------
기여도: [11. 16.  4. 16. 21.  9. 22.]
[[0.009 0.009 0.009]]
[[0.245 0.295 0.359]
 [0.391 0.256 0.149]
 [0.231 0.273 0.267]
 [0.696 0.544 0.521]
 [0.291 0.504 0.551]
 [0.538 0.675 0.756]
 [0.1   0.094 0.078]]
[[0.50989411 0.47595483 0.43074819 1.01011781 0.78632881 1.13199205
  0.14236221]]
6
-----------
[[0.009 0.009 0.009]]
[[0.312 0.325 0.354]
 [0.369 0.287 0.201]
 [0.22  0.229 0.229]
 [0.666 0.489 0.413]
 [0.382 0.474 0.484]
 [0.562 0.734 0.662]
 [0.11  0.099 0.103]]
[[0.55739573 0.49370842 0.37592685 0.90844097 0.76221978 1.121536
  0.16473312]]
6
-----------
[[0.009 0.009 0.009]]
[[0.316 0.329 0.345]
 [0.371 0.294 0.215]
 [0.215 0.216 0.223]
 [0.674 0.483 0.375]
 [0.398 0.469 0.48 ]
 [0.562 0.734 0.662]
 [0.109 0.097 0.09

기여도: [18. 10.  6. 28. 12. 24.  2.]
[[0.009 0.009 0.009]]
[[0.384 0.598 0.348]
 [0.393 0.202 0.202]
 [0.478 0.348 0.285]
 [0.305 0.288 0.248]
 [0.715 0.605 0.305]
 [0.434 0.403 0.22 ]
 [0.335 0.306 0.305]]
[[0.77618748 0.47111994 0.64113805 0.47178173 0.97018967 0.61675117
  0.53113181]]
1
-----------
[[0.009 0.009 0.009]]
[[0.472 0.629 0.506]
 [0.295 0.15  0.133]
 [0.486 0.383 0.334]
 [0.189 0.2   0.175]
 [0.775 0.675 0.479]
 [0.403 0.42  0.244]
 [0.302 0.326 0.409]]
[[0.91966189 0.34213009 0.68777176 0.31054307 1.11857588 0.61593993
  0.58850489]]
3
-----------
[[0.009 0.009 0.009]]
[[0.505 0.642 0.495]
 [0.316 0.167 0.15 ]
 [0.531 0.397 0.333]
 [0.162 0.176 0.154]
 [0.811 0.704 0.507]
 [0.4   0.384 0.251]
 [0.282 0.323 0.418]]
[[0.93962812 0.37295308 0.72663884 0.26892936 1.17227684 0.59335487
  0.58344323]]
3
-----------
[[0.009 0.009 0.009]]
[[0.525 0.64  0.496]
 [0.305 0.182 0.149]
 [0.559 0.417 0.341]
 [0.139 0.157 0.132]
 [0.832 0.727 0.518]
 [0.398 0.375 0.263]
 [0.267 0.308 0.

,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,...,아파트1-102-901,아파트1-103-1905,아파트1-103-503,아파트1-103-1504,아파트1-103-606,아파트1-103-903,아파트1-103-1106,아파트1-103-705,아파트1-103-1505,아파트1-103-406
0,4.0,3.0,1.0,5.0,5.0,5.0,3.0,4.0,4.0,3.0,...,4.0,3.0,4.0,4.0,6.0,6.0,3.0,3.0,3.0,4.0
1,3.0,4.0,2.0,1.0,1.0,4.0,4.0,6.0,4.0,4.0,...,4.0,2.0,5.0,4.0,5.0,6.0,2.0,2.0,2.0,5.0
2,0.0,3.0,3.0,3.0,3.0,0.0,6.0,0.0,0.0,0.0,...,3.0,2.0,2.0,1.0,5.0,1.0,5.0,0.0,1.0,1.0
3,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,4.0,2.0,...,2.0,4.0,0.0,3.0,3.0,3.0,2.0,0.0,5.0,3.0
4,2.0,6.0,6.0,6.0,6.0,2.0,0.0,1.0,3.0,2.0,...,0.0,4.0,0.0,1.0,3.0,0.0,6.0,2.0,2.0,3.0
5,2.0,6.0,6.0,6.0,6.0,2.0,0.0,2.0,5.0,3.0,...,1.0,4.0,1.0,3.0,3.0,1.0,5.0,2.0,3.0,3.0
6,2.0,6.0,6.0,6.0,6.0,2.0,3.0,4.0,0.0,4.0,...,4.0,4.0,0.0,0.0,0.0,4.0,4.0,1.0,1.0,4.0
7,5.0,3.0,3.0,3.0,3.0,1.0,5.0,2.0,6.0,5.0,...,0.0,5.0,4.0,5.0,4.0,5.0,5.0,1.0,5.0,5.0


In [643]:
_hc = households_cluster.copy()
_test = list()

for col in _hc:
    _hc_info = _hc[col].copy()
    contributions = [cluster_info[division_round][1][int(_)] for division_round,_ in enumerate(_hc_info)]
    _test.append(contributions.copy())

contribution_df = pd.DataFrame(_test).T
contribution_df.columns = df.columns

contribution_df.head()

,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,...,아파트1-102-901,아파트1-103-1905,아파트1-103-503,아파트1-103-1504,아파트1-103-606,아파트1-103-903,아파트1-103-1106,아파트1-103-705,아파트1-103-1505,아파트1-103-406
0,13.0,8.0,14.0,4.0,4.0,4.0,8.0,13.0,13.0,8.0,...,13.0,8.0,13.0,13.0,19.0,19.0,8.0,8.0,8.0,13.0
1,17.0,9.0,12.0,3.0,3.0,9.0,9.0,14.0,9.0,9.0,...,9.0,12.0,19.0,9.0,19.0,14.0,12.0,12.0,12.0,19.0
2,8.0,3.0,3.0,3.0,3.0,8.0,13.0,8.0,8.0,8.0,...,3.0,26.0,26.0,16.0,13.0,16.0,13.0,8.0,16.0,16.0
3,4.0,4.0,4.0,4.0,4.0,11.0,11.0,11.0,21.0,4.0,...,4.0,21.0,11.0,16.0,16.0,16.0,4.0,11.0,9.0,16.0
4,8.0,4.0,4.0,4.0,4.0,8.0,12.0,12.0,21.0,8.0,...,12.0,17.0,12.0,12.0,21.0,12.0,4.0,8.0,8.0,21.0


In [644]:
cols = df.columns
min_col = cols[contribution_df.mean().round().argmin()]
max_col = cols[contribution_df.mean().round().argmax()]

In [658]:
contribution_df.mean().sort_values

<bound method Series.sort_values of 아파트1-104-1206     9.625
아파트1-104-303      4.500
아파트1-104-1307     5.625
아파트1-104-1208     3.250
아파트1-104-408      3.250
                  ...  
아파트1-103-903     14.250
아파트1-103-1106    10.125
아파트1-103-705      9.125
아파트1-103-1505    12.250
아파트1-103-406     16.500
Length: 108, dtype: float64>

In [651]:
print("최소기여가구 : {}".format(min_col))
print("최대기여가구 : {}".format(max_col))

최소기여가구 : 아파트1-104-1208
최대기여가구 : 아파트1-103-1006


In [652]:
_min_col = cols[df.sum().argmin()]
_max_col = cols[df.sum().argmax()]

print(_min_col)
print(_max_col)

아파트1-104-1208
아파트1-103-703


In [647]:
df[_min_col].sum()

0.21899999999999997

In [648]:
df[min_col].sum()

0.21899999999999997

In [653]:
df[_max_col].sum()

15.629999999999999

In [654]:
df[max_col].sum()

15.342999999999998

In [655]:
df

,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,...,아파트1-102-901,아파트1-103-1905,아파트1-103-503,아파트1-103-1504,아파트1-103-606,아파트1-103-903,아파트1-103-1106,아파트1-103-705,아파트1-103-1505,아파트1-103-406
2018-05-01 00:00:00,0.341,0.121,0.245,0.009,0.085,0.154,0.161,0.311,0.358,0.177,...,0.372,0.181,0.322,0.374,0.571,0.453,0.161,0.127,0.259,0.364
2018-05-01 01:00:00,0.337,0.186,0.285,0.010,0.071,0.127,0.179,0.225,0.206,0.178,...,0.245,0.238,0.292,0.382,0.327,0.352,0.172,0.133,0.234,0.286
2018-05-01 02:00:00,0.324,0.214,0.487,0.009,0.061,0.139,0.182,0.221,0.150,0.152,...,0.245,0.250,0.287,0.320,0.359,0.366,0.242,0.219,0.197,0.368
2018-05-01 03:00:00,0.319,0.250,0.300,0.009,0.067,0.132,0.172,0.274,0.142,0.205,...,0.177,0.223,0.329,0.170,0.322,0.267,0.269,0.164,0.229,0.414
2018-05-01 04:00:00,0.235,0.199,0.107,0.009,0.047,0.131,0.153,0.255,0.139,0.169,...,0.129,0.171,0.301,0.168,0.351,0.255,0.170,0.132,0.198,0.318
2018-05-01 05:00:00,0.169,0.138,0.301,0.009,0.039,0.156,0.186,0.273,0.171,0.115,...,0.221,0.238,0.308,0.196,0.356,0.253,0.221,0.346,0.228,0.329
2018-05-01 06:00:00,0.200,0.137,0.205,0.009,0.036,0.134,0.201,0.198,0.137,0.134,...,0.104,0.277,0.257,0.295,0.571,0.390,0.344,0.220,0.319,0.329
2018-05-01 07:00:00,0.171,0.117,0.123,0.009,0.038,0.185,0.333,0.252,0.137,0.374,...,0.101,0.278,0.689,0.383,0.382,0.473,0.377,0.239,0.560,0.437
2018-05-01 08:00:00,0.170,0.121,0.064,0.009,0.037,0.222,0.390,0.208,0.301,0.272,...,0.130,1.252,0.651,0.438,0.322,0.480,0.294,0.331,0.445,0.483
2018-05-01 09:00:00,0.172,0.228,0.063,0.009,0.055,0.214,0.322,0.244,0.400,0.194,...,0.125,0.641,0.334,0.426,0.469,0.540,0.148,0.357,0.305,0.486


In [622]:
contribution_df[_min_col]

0     3.0
1    18.0
2     3.0
3    13.0
4    13.0
5    19.0
6     7.0
7    17.0
Name: 아파트1-104-1208, dtype: float64

In [623]:
contribution_df[min_col]

0    12.0
1    12.0
2    12.0
3     3.0
4     3.0
5     9.0
6    14.0
7     3.0
Name: 아파트1-101-301, dtype: float64

In [624]:
households_cluster[_min_col]

0    3.0
1    1.0
2    3.0
3    3.0
4    3.0
5    5.0
6    2.0
7    4.0
Name: 아파트1-104-1208, dtype: float64

In [641]:
cluster_info[1][1]

array([28., 18.,  8.,  3., 17., 13., 12.])

In [584]:
households_cluster[min_col]

0    1.0
1    1.0
2    4.0
3    0.0
4    2.0
5    2.0
6    3.0
7    3.0
Name: 아파트1-102-1302, dtype: float64

In [590]:
cluster_info[0][0]

array([[0.198, 0.31 , 0.49 ],
       [0.25 , 0.24 , 0.223],
       [0.583, 0.531, 0.462],
       [0.872, 0.648, 0.511],
       [0.162, 0.153, 0.162],
       [0.067, 0.063, 0.056],
       [0.419, 0.314, 0.297]])

In [594]:
euc(cluster_info[2][0], np.expand_dims(df[min_col][6:9].values, axis=0)).argmin()

1

In [560]:
print(cluster_info[2][0][5])
cluster_info[2][1][5]

[0.748 0.613 0.556]


27.0

In [559]:
print(cluster_info[2][0][])
cluster_info[2][1][0]

[0.538 0.25  0.226]


14.0